In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import pandas as pd

In [2]:
pmd_with_rsids_df = pd.read_csv(home_dir+"data/pmd/pmd_with_rsids.tsv", sep="\t")
pmd_with_rsids_df["snp_id"] = pmd_with_rsids_df["snp_id"].apply(lambda x: "rs"+str(int(x)))
print(pmd_with_rsids_df.shape)
print(pmd_with_rsids_df.columns)

(2783, 55)
Index(['mut_id', 'md5', 'pmd_id', 'nr', 'authors', 'journal', 'title',
       'medline', 'crossref', 'uniprot_id', 'ensembl_id', 'other_ref',
       'protein', 'source', 'expression_sys', 'mut_PMD', 'mut_real',
       'function', 'fb', 'structure', 'strB', 'stability', 'staB',
       'expression', 'eB', 'transport', 'tB', 'maturation', 'mB', 'disease',
       'dB', 'uni_real', 'uni_realid', 'uni_start', 'uni_finish', 'uniq_start',
       'uniq_finish', 'uni_loc', 'ens_real', 'ens_organism', 'ens_start',
       'ens_finish', 'ensq_start', 'ensq_finish', 'ens_loc', 'pos_real',
       'mt_real', 'taxid', 'snp_id', 'chrom', 'chrom_pos', 'variation',
       'variant_type', 'ref_allele', 'alt_allele'],
      dtype='object')


In [3]:
pmd_dbnsfp_df = pd.read_csv(home_dir+"data/pmd/pmd_with_dbnsfp.tsv", sep="\t")
pmd_dbnsfp_df.rename(columns={'#chr':'chrom', 'pos(1-based)':'chrom_pos', 'ref':'ref_allele', 'alt':'alt_allele', 'rs_dbSNP':'snp_id'}, inplace=True)
pmd_dbnsfp_df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
pmd_dbnsfp_df["snp_id"].replace(".", None, inplace=True)
print(pmd_dbnsfp_df.shape)
print(pmd_dbnsfp_df.columns)

# pmd_dbnsfp_df["snp_id"]

(7310, 35)
Index(['mut_id', 'md5', 'pmd_id', 'nr', 'pmd_nr_id', 'crossref', 'uniprot_id',
       'protein', 'source', 'mut_PMD', 'mut_real', 'wt', 'mut', 'prot_pos',
       'chrom', 'chrom_pos', 'ref_allele', 'alt_allele', 'snp_id', 'genename',
       'ensembl_id', 'function', 'function_summarized', 'functional_effect',
       'CADD_raw', 'MetaRNN_score', 'MVP_score', 'Polyphen2_HVAR_score',
       'REVEL_score', 'SIFT_score', 'integrated_fitCons_score',
       'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic', 'seq'],
      dtype='object')


In [4]:
columns = ['mut_id', 'md5', 'pmd_id', 'nr', 'mut_real', 'chrom', 'chrom_pos', 'ref_allele', 'alt_allele', 'function', 'source', 'crossref', 'snp_id']
x = pmd_with_rsids_df[columns]
y = pmd_dbnsfp_df[columns]
z = pd.merge(x, y, how='outer', on=columns)
z.drop_duplicates(keep="first", inplace=True, ignore_index=True)
z.value_counts()

for col in columns:
    print(col, z[pd.isna(z[col])].shape)

pmd_clean_df = z.copy(deep=True)
print(pmd_clean_df.shape)

mut_id (0, 13)
md5 (0, 13)
pmd_id (0, 13)
nr (0, 13)
mut_real (0, 13)
chrom (0, 13)
chrom_pos (0, 13)
ref_allele (0, 13)
alt_allele (0, 13)
function (0, 13)
source (0, 13)
crossref (0, 13)
snp_id (4999, 13)
(8860, 13)


In [5]:
# merging with raw PMD-with-rsids data
data_merged_with_pmd_raw = pd.merge(left=pmd_clean_df, right=pmd_with_rsids_df, how="left", on=columns)
print(data_merged_with_pmd_raw.shape)
print(data_merged_with_pmd_raw.columns)

(8860, 55)
Index(['mut_id', 'md5', 'pmd_id', 'nr', 'mut_real', 'chrom', 'chrom_pos',
       'ref_allele', 'alt_allele', 'function', 'source', 'crossref', 'snp_id',
       'authors', 'journal', 'title', 'medline', 'uniprot_id', 'ensembl_id',
       'other_ref', 'protein', 'expression_sys', 'mut_PMD', 'fb', 'structure',
       'strB', 'stability', 'staB', 'expression', 'eB', 'transport', 'tB',
       'maturation', 'mB', 'disease', 'dB', 'uni_real', 'uni_realid',
       'uni_start', 'uni_finish', 'uniq_start', 'uniq_finish', 'uni_loc',
       'ens_real', 'ens_organism', 'ens_start', 'ens_finish', 'ensq_start',
       'ensq_finish', 'ens_loc', 'pos_real', 'mt_real', 'taxid', 'variation',
       'variant_type'],
      dtype='object')


In [6]:
# merging with PMD dbnsfp data
data_merged_with_pmd_dbnsfp = pd.merge(left=data_merged_with_pmd_raw, right=pmd_dbnsfp_df, how="left", on=columns)
print(data_merged_with_pmd_dbnsfp.shape)
print(data_merged_with_pmd_dbnsfp.columns)

(8860, 77)
Index(['mut_id', 'md5', 'pmd_id', 'nr', 'mut_real', 'chrom', 'chrom_pos',
       'ref_allele', 'alt_allele', 'function', 'source', 'crossref', 'snp_id',
       'authors', 'journal', 'title', 'medline', 'uniprot_id_x',
       'ensembl_id_x', 'other_ref', 'protein_x', 'expression_sys', 'mut_PMD_x',
       'fb', 'structure', 'strB', 'stability', 'staB', 'expression', 'eB',
       'transport', 'tB', 'maturation', 'mB', 'disease', 'dB', 'uni_real',
       'uni_realid', 'uni_start', 'uni_finish', 'uniq_start', 'uniq_finish',
       'uni_loc', 'ens_real', 'ens_organism', 'ens_start', 'ens_finish',
       'ensq_start', 'ensq_finish', 'ens_loc', 'pos_real', 'mt_real', 'taxid',
       'variation', 'variant_type', 'pmd_nr_id', 'uniprot_id_y', 'protein_y',
       'mut_PMD_y', 'wt', 'mut', 'prot_pos', 'genename', 'ensembl_id_y',
       'function_summarized', 'functional_effect', 'CADD_raw', 'MetaRNN_score',
       'MVP_score', 'Polyphen2_HVAR_score', 'REVEL_score', 'SIFT_score',
       '

In [7]:
method_score_cols = ["CADD_raw", "MetaRNN_score", "MVP_score", "Polyphen2_HVAR_score", "REVEL_score", "SIFT_score", "integrated_fitCons_score", "phyloP17way_primate", "phastCons17way_primate", "bStatistic"]
x_columns = columns + method_score_cols
data_merged_with_pmd_dbnsfp = data_merged_with_pmd_dbnsfp[x_columns]
print(data_merged_with_pmd_dbnsfp.shape)
print(data_merged_with_pmd_dbnsfp.columns)

(8860, 23)
Index(['mut_id', 'md5', 'pmd_id', 'nr', 'mut_real', 'chrom', 'chrom_pos',
       'ref_allele', 'alt_allele', 'function', 'source', 'crossref', 'snp_id',
       'CADD_raw', 'MetaRNN_score', 'MVP_score', 'Polyphen2_HVAR_score',
       'REVEL_score', 'SIFT_score', 'integrated_fitCons_score',
       'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic'],
      dtype='object')


In [8]:
# loading PMD seq table
seq_pmd_df = pd.read_csv(home_dir+"data/SNPdbe/seqs_PMD.csv", sep="\t;") # PMD: from SNPdbe # (127565, 48)
seq_pmd_df.drop(columns=["taxid", "md5"], inplace=True)
print(seq_pmd_df.shape) 
print(seq_pmd_df.columns)

/tmp/ipykernel_2896187/2731603977.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  seq_pmd_df = pd.read_csv(home_dir+"data/SNPdbe/seqs_PMD.csv", sep="\t;") # PMD: from SNPdbe # (127565, 48)


(53124, 3)
Index(['pmd_id', 'nr', 'seq'], dtype='object')


In [9]:
# merging with seq information
data_merged_with_seq = pd.merge(left=data_merged_with_pmd_dbnsfp, right=seq_pmd_df, on=["pmd_id", "nr"], how="inner")
print(data_merged_with_seq.shape) 
print(data_merged_with_seq.columns)

(8860, 24)
Index(['mut_id', 'md5', 'pmd_id', 'nr', 'mut_real', 'chrom', 'chrom_pos',
       'ref_allele', 'alt_allele', 'function', 'source', 'crossref', 'snp_id',
       'CADD_raw', 'MetaRNN_score', 'MVP_score', 'Polyphen2_HVAR_score',
       'REVEL_score', 'SIFT_score', 'integrated_fitCons_score',
       'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic', 'seq'],
      dtype='object')


In [10]:
# extract functional annotation
def summarize_functional_effect(mut_function):
    # print(mut_function)
    mut_function = str(mut_function)
    effects = []
    effect = ""
    flag = False
    for i, ch in enumerate(mut_function):
        if ch=='[':
            flag = True
            continue
        if flag and ch in ['-', '+'] and mut_function[i+1] in ['-', '+', ']', ' ']:
            effect += ch
        elif flag and ch == '=':
            effects.append('=')
        elif flag and ch == '0' and mut_function[i+1]=="]":
            effects.append('0')
            
        if ch=="]":
            flag = False
            if len(effect)>0: effects.append(effect)
            effect=""
    
    if len(effects)>0: return effects
    else: return ""

def get_functional_effect(effects):
    counts = [] 
    functional_effect = ""
    for ef in effects:
        if "0" in ef:
            return "Knock-out"
        elif "=" in ef:
            functional_effect = "No-effect"
        else:
            counts.append(len(ef)-2)

    
    if len(counts)==0 and functional_effect=="No-effect": return "No-effect"
    elif max(counts)<3: return "Effect"
    else: return "Knock-out"

df = data_merged_with_seq.copy(deep=True)
df["wt"] = df["mut_real"].apply(lambda x: x[0]) # 1-letter amino acid
df["mut"] = df["mut_real"].apply(lambda x: x[-1])
df["prot_pos"] = df["mut_real"].apply(lambda x: int(x[1:-1])) # mutation position is 1-indexed

df["function_summarized"] = df["function"].apply(summarize_functional_effect)
df = df[df["function_summarized"].apply(lambda x:  len(x)>0)] # removing rows that does not have any functional effect annotations
df["functional_effect"] = df["function_summarized"].apply(get_functional_effect)

data_filered_on_func = df.copy(deep=True)
print(data_filered_on_func.shape) 
print(data_filered_on_func.columns)

(8851, 29)
Index(['mut_id', 'md5', 'pmd_id', 'nr', 'mut_real', 'chrom', 'chrom_pos',
       'ref_allele', 'alt_allele', 'function', 'source', 'crossref', 'snp_id',
       'CADD_raw', 'MetaRNN_score', 'MVP_score', 'Polyphen2_HVAR_score',
       'REVEL_score', 'SIFT_score', 'integrated_fitCons_score',
       'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic', 'seq',
       'wt', 'mut', 'prot_pos', 'function_summarized', 'functional_effect'],
      dtype='object')


In [11]:
# filter on human
df = data_filered_on_func.copy(deep=True)
df = df[df["source"].apply(lambda x: True if "Human" in str(x) else False)] # only keeping human variants
print("after removing non-human entries:", df.shape)

data_filered_on_human = df.copy(deep=True)

after removing non-human entries: (8832, 29)


In [12]:
# filter on seq-len
df = data_filered_on_human.copy(deep=True)
df["pmd_nr_id"] = df["pmd_id"]+df["nr"].apply(lambda x: "_"+str(int(x)))
sequences_df = df[['pmd_nr_id', 'crossref', 'seq']].drop_duplicates(keep="first")
print("#-of sequences b4 seq-len filter", sequences_df.shape[0])
protid_seq_tuple_list = [(tuple.pmd_nr_id, tuple.seq) for tuple in sequences_df.itertuples() if len(str(tuple.seq))<=1022]
print("#-of sequences after seq-len filter", len(protid_seq_tuple_list))

new_protein_acc_list = list(zip(*protid_seq_tuple_list))[0]
df = df[df["pmd_nr_id"].isin(new_protein_acc_list)]
print("after filtering on seq-len <=1022:", df.shape)
print(df["functional_effect"].value_counts())

data_filered_on_seqlen = df.copy(deep=True)
data_filered_on_seqlen.columns

#-of sequences b4 seq-len filter 2235
#-of sequences after seq-len filter 2198
after filtering on seq-len <=1022: (8582, 30)
Effect       4633
Knock-out    1981
No-effect    1968
Name: functional_effect, dtype: int64


Index(['mut_id', 'md5', 'pmd_id', 'nr', 'mut_real', 'chrom', 'chrom_pos',
       'ref_allele', 'alt_allele', 'function', 'source', 'crossref', 'snp_id',
       'CADD_raw', 'MetaRNN_score', 'MVP_score', 'Polyphen2_HVAR_score',
       'REVEL_score', 'SIFT_score', 'integrated_fitCons_score',
       'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic', 'seq',
       'wt', 'mut', 'prot_pos', 'function_summarized', 'functional_effect',
       'pmd_nr_id'],
      dtype='object')

In [13]:
df = data_filered_on_seqlen.copy(deep=True)
df.to_csv(home_dir+"models/aa_common/datasets_pmd_analysis/pmd.tsv", sep="\t", index=False)
sequences_df = df[['pmd_nr_id', 'crossref', 'seq']].drop_duplicates(keep="first")
print("#-of sequences", sequences_df.shape)

out_fasta_filepath = home_dir+"models/aa_common/datasets_pmd_analysis/pmd_sequences.fasta"
out_fasta_file_handle = open(out_fasta_filepath, "w")

for tuple in sequences_df.itertuples():
    out_fasta_file_handle.write(f">{tuple.pmd_nr_id} | {tuple.crossref}\n")
    out_fasta_file_handle.write(f"{tuple.seq}\n")
    # break
out_fasta_file_handle.close()  

#-of sequences (2198, 3)


### tests

In [14]:
# 'mut_id', 'md5', 'pmd_id', 'nr', 'mut_real', 'chrom', 'chrom_pos', 'ref_allele', 'alt_allele', 'function', 'source', 'crossref', 'snp_id'
print("#-of protein variants: ", df[['mut_id', 'md5', 'pmd_id', 'nr', 'mut_real']].drop_duplicates(keep="first").shape[0])
print("#-of chromosomal variants: ", df[['chrom', 'chrom_pos', 'ref_allele', 'alt_allele']].drop_duplicates(keep="first").shape[0])
print("#-of protein sequences: ", df[['pmd_id', 'nr']].drop_duplicates(keep="first").shape[0])
print("#-of rows not mapped to snp-ids: ", df[pd.isna(df["snp_id"])].shape[0])
print("#-of rows mapped to snp-ids: ", df[~pd.isna(df["snp_id"])].shape[0])
print("total: ", df.shape[0])
df["functional_effect"].value_counts()


#-of protein variants:  6663
#-of chromosomal variants:  6475
#-of protein sequences:  2198
#-of rows not mapped to snp-ids:  4999
#-of rows mapped to snp-ids:  3583
total:  8582


Effect       4633
Knock-out    1981
No-effect    1968
Name: functional_effect, dtype: int64

In [15]:
z = data_filered_on_seqlen
for col in z.columns:
    print(col, z[pd.isna(z[col])].shape)

mut_id (0, 30)
md5 (0, 30)
pmd_id (0, 30)
nr (0, 30)
mut_real (0, 30)
chrom (0, 30)
chrom_pos (0, 30)
ref_allele (0, 30)
alt_allele (0, 30)
function (0, 30)
source (0, 30)
crossref (0, 30)
snp_id (4999, 30)
CADD_raw (1272, 30)
MetaRNN_score (1272, 30)
MVP_score (1272, 30)
Polyphen2_HVAR_score (1272, 30)
REVEL_score (1272, 30)
SIFT_score (1272, 30)
integrated_fitCons_score (1272, 30)
phyloP17way_primate (1272, 30)
phastCons17way_primate (1272, 30)
bStatistic (1272, 30)
seq (0, 30)
wt (0, 30)
mut (0, 30)
prot_pos (0, 30)
function_summarized (0, 30)
functional_effect (0, 30)
pmd_nr_id (0, 30)
